In [1]:
import cv2 
import os
import numpy as np

from util import get_parking_spots_bboxes, empty_or_not, calc_diff

c:\Users\user\anaconda3\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.2.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [2]:
# Загрузить маску
mask = os.path.join('.', 'mask_1920_1080.png')
mask = cv2.imread(mask, 0)

In [3]:
# В переменную spots кладутся координаты мест для парковки
connected_components = cv2.connectedComponentsWithStats(mask, 4, cv2.CV_32S)
spots = get_parking_spots_bboxes(connected_components)
print(spots[0])

[533, 98, 65, 35]


In [4]:
# Переменные для предиктов
step = 60 # Частота предикта (раз в 60 сек)
spots_status = [None for j in spots] # Список всех статусов мест парковок
diffs = [None for j in spots] # Здесь хранятся разницы между предыдущим и текущим кадром. При изменении будет делать предикт
frame_num = 0 # Счётчик кадров
previous_frame = None # Хранит данные о предыдущем кадре, duh

In [5]:
# Загрузить видео
video_path = os.path.join('.', 'data', 'parking_1920_1080_loop.mp4')
video = cv2.VideoCapture(video_path)

# Проиграть видео
ret = True
while ret:
    ret, frame = video.read()

    if frame_num % step == 0 and previous_frame is not None:
        for spot_index, spot in enumerate(spots): # Цикл заполнения списка diffs разницей между предыдущим кадром и текущим
            x1, y1, w, h = spot
            spot_crop = frame[y1:y1+h, x1:x1+w, :]
            diffs[spot_index] = calc_diff(spot_crop, previous_frame[y1:y1+h, x1:x1+w, :])

    if frame_num % step == 0:
        if previous_frame is None:
            arr_ = range(len(spots))
        else:
            arr_ = [j for j in np.argsort(diffs) if diffs[j] / np.max(diffs) > 0.4] 
        # Цикл предикта для каждого места парковки. 
        # Работает, только на кадрах, разница средних которых попадает в 
        # большие 60% diffs[j] / np.max(diffs), то есть выбросы. 
        for spot_index in arr_:
            spot = spots[spot_index]
            x1, y1, w, h = spot
            spot_crop = frame[y1:y1+h, x1:x1+w, :]
            spot_status = empty_or_not(spot_crop) # Обученная модель определяет статус парковочного места
            spots_status[spot_index] = spot_status 

    if frame_num % step == 0:
        previous_frame = frame.copy() # Сохраняет в previous_frame текущий кадр
            
    for spot_index, spot in enumerate(spots): # Цикл рисования прямоугольников в соответствии с предиктом
        spot_status = spots_status[spot_index]
        x1, y1, w, h = spots[spot_index]
        if spot_status:
            frame = cv2.rectangle(frame, (x1, y1), (x1+w, y1+h), (0, 255, 0) ,2)
        else:
            frame = cv2.rectangle(frame, (x1, y1), (x1+w, y1+h), (0, 0, 255) ,2)
    
    # Счётчик доступных мест с фоном
    cv2.rectangle(frame, (80, 20), (660, 80), (0, 0, 0), -1)
    cv2.putText(frame, f'Available parking places: {str(sum(spots_status))}/{str(len(spots_status))}', (100,60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
    cv2.imshow('frame', frame)
    if  cv2.waitKey(25) % 0xFF == ord('q'):
        break
        
    frame_num += 1
        
video.release()
cv2.destroyAllWindows()